<a href="https://colab.research.google.com/github/hand-e-fr/OpenHosta/blob/doc/docs/openhosta_agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# OpenHosta Agent with GPT4

This colab demonstrate simple use cases of OpenHosta. You need an OpenAI key to run it

## Basic Usage of AI Agents with OpenHosta

In [1]:
!pip install OpenHosta

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.6/61.6 kB 3.3 MB/s eta 0:00:00


### Configure the LLM that you want to use

In [1]:
from OpenHosta import config

# You can skip this line if you plan to use the local ollama instace of phi-4
config.set_default_apiKey("...") # Ask me one through LinkedIn, I will provide one for testing purpose!

## Agent structure

Ask a question then select best function to answer

Within Google colab you must use asynchrone version of ̀`emulate`. If you want to reproduce this without coroutines, remove `.asynchrone` in the import line and all `await` and `async`

### Emulate functions using the seleted LLM

In [2]:
from OpenHosta.asynchrone import emulate

In [3]:
# Mix of python and Openhosta functions that the agent will use
async def add(a:float, b:float)->float:
  """
  add two float numbers
  """
  return a+b

async def number_to_string(number:float)->str:
  """
  Convert a number to a string that represent the number in letters
  """
  return await emulate()

async def string_to_number(input_string:str)->float:
  """
  Convert input_string to a number
  """
  return await emulate()

In [4]:
# Agent main router
from typing import Dict, Literal, List, Tuple

Actions = {
    "add": add,
    "number_to_string": number_to_string,
    "string_to_number": string_to_number
}

ActionType = Literal["add", "number_to_string", "string_to_number", "Done"]


In [5]:
import inspect

annotations = {f:inspect.get_annotations(f) for f in Actions.values()}

In [6]:
async def find_best_step_to_execute(stack:List[Tuple[ActionType, list, str]], actions:dict, target)->Tuple[ActionType, list]:
  """
  Select the best action to get elements needed to achieve target.

  You first print a strategy based on possible actions,
  Then you look at already executed actions in stack,
  Then you decide of the next action to take and its parameters

  :param stack: a list of already taken actions with (ActionType, list of params, Action returned value)
  :param actions: a dictionarry of possible tools
  :param target: the overall target objective.

  :return: The next action to take and its parameters. Return Done if there is no more action to take
  """
  return await emulate()

### Agent main loop

Skip this part if you do not have OpenAI API Key

In [8]:
request = "can you add twenty two and hundred thousant fifity five and print the result in letters"

stack=[]
output = ""

Done=False
while not Done:
  next_action, params = await find_best_step_to_execute(stack, annotations, request)
  if next_action == "Done":
    Done=True
  else:
    next_function = Actions[next_action]
    print(f"Executing {next_action} with params {params}")
    output = await next_function(*params)
    stack.append([next_action, params, output])

print(output)


Executing string_to_number with params ['twenty two']
Executing number_to_string with params [105077.0]
Executing add with params [22.0, 105077.0]
Executing string_to_number with params ['one hundred thousand fifty five']
Executing number_to_string with params [105077.0]
Executing number_to_string with params [105077.0]
one hundred five thousand seventy-seven


# OpenHosta Agent with a Local Phi-4 LLM

Redo the same but with a local phi-4 instance

In [9]:
!apt install -y screen
!curl -fsSL https://ollama.com/install.sh | sh
!screen -dmS ollama ollama serve


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
Suggested packages:
  byobu | screenie | iselect ncurses-term
The following NEW packages will be installed:
  screen
0 upgraded, 1 newly installed, 0 to remove and 19 not upgraded.
Need to get 672 kB of archives.
After this operation, 1,029 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/main amd64 screen amd64 4.9.0-1 [672 kB]
Fetched 672 kB in 1s (596 kB/s)
Selecting previously unselected package screen.
(Reading database ... 124926 files and directories currently installed.)
Preparing to unpack .../screen_4.9.0-1_amd64.deb ...
Unpacking screen (4.9.0-1) ...
Setting up screen (4.9.0-1) ...
Processing triggers for man-db (2.10.2-1) ...
>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
############################################################################################# 100.0%
>>> Creating ollama user...
>>> Adding ollama u

In [13]:
!echo Downloading model. This can take 5min ...
!sleep 1
#!ollama run phi4 hello --verbose  2>&1 | grep -E ":"
!ollama run qwen2.5-coder:3b hello --verbose  2>&1 | grep -E ":"

total duration:       3.638193238s
load duration:        2.707758378s
prompt eval count:    30 token(s)
prompt eval duration: 302ms
prompt eval rate:     99.34 tokens/s
eval count:           37 token(s)
eval duration:        627ms
eval rate:            59.01 tokens/s


In [14]:
from OpenHosta import config
# Use Microsoft local Phi-4 through ollama
my_model=config.Model(
     base_url="http://localhost:11434/v1/chat/completions",
     #model="phi4", api_key="none", timeout=120
     model="qwen2.5-coder:3b", api_key="none", timeout=120
 )
config.set_default_model(my_model)


In [20]:
request = "can you add twenty two and hundred thousant fifity five and print the result in letters"

stack=[]
output = ""

Done=False
while not Done:
  next_action, params = await find_best_step_to_execute(stack, annotations, request)
  if next_action == "Done":
    Done=True
  else:
    next_function = Actions[next_action]
    print(f"Executing {next_action} with params {params}")
    output = await next_function(*params)
    stack.append([next_action, params, output])

print(output)


ValueError: not enough values to unpack (expected 2, got 1)